# Script Generators

> Generate complete keyboard navigation JavaScript from configuration.

In [ ]:
#| default_exp js.generators

In [ ]:
#| export
from __future__ import annotations

from cjm_fasthtml_keyboard_navigation.core.manager import ZoneManager
from cjm_fasthtml_keyboard_navigation.js.utils import (
    js_config_from_dict,
    js_all_utils
)

## Zone State Management

In [ ]:
#| export
def js_zone_state() -> str: # JavaScript state and getter/setter code
    """Generate JavaScript code for zone state management."""
    return '''
// === State ===
let activeZoneId = cfg.initialZoneId;
let focusIndices = {};
let currentMode = cfg.defaultMode;

// Initialize focus indices
for (const zone of cfg.zones) {
    focusIndices[zone.id] = zone.initialIndex || 0;
}

// === Zone Getters ===
function getZoneConfig(zoneId) {
    return cfg.zones.find(z => z.id === zoneId);
}

function getZoneElement(zoneId) {
    return document.getElementById(zoneId);
}

function getZoneItems(zoneId) {
    const zone = getZoneConfig(zoneId);
    if (!zone || !zone.itemSelector) return [];
    const container = getZoneElement(zoneId);
    if (!container) return [];
    return Array.from(container.querySelectorAll(zone.itemSelector));
}

function getFocusedItem(zoneId) {
    const items = getZoneItems(zoneId);
    const idx = focusIndices[zoneId] || 0;
    return items[idx] || null;
}
'''.strip()

## Focus Management

In [ ]:
#| export
def js_focus_management() -> str: # JavaScript focus management code
    """Generate JavaScript code for focus management."""
    return '''
// === Focus Management ===
function clearItemFocus(zoneId) {
    const zone = getZoneConfig(zoneId);
    if (!zone) return;
    
    const items = getZoneItems(zoneId);
    for (const item of items) {
        removeFocusRing(item, zone.itemFocusClasses);
        item.setAttribute(zone.itemFocusAttribute, 'false');
    }
}

function clearZoneFocus(zoneId) {
    const zone = getZoneConfig(zoneId);
    if (!zone) return;
    
    const container = getZoneElement(zoneId);
    if (container) {
        removeFocusRing(container, zone.zoneFocusClasses);
    }
}

function setItemFocus(zoneId, index, triggerCallbacks = true) {
    const zone = getZoneConfig(zoneId);
    if (!zone) return;
    
    const items = getZoneItems(zoneId);
    if (items.length === 0) return;
    
    // Clamp index
    const clampedIdx = Math.max(0, Math.min(index, items.length - 1));
    focusIndices[zoneId] = clampedIdx;
    
    // Clear existing focus
    clearItemFocus(zoneId);
    
    // Apply focus to new item
    const item = items[clampedIdx];
    if (item) {
        addFocusRing(item, zone.itemFocusClasses);
        item.setAttribute(zone.itemFocusAttribute, 'true');
        scrollToElement(item, { behavior: zone.scrollBehavior, block: zone.scrollBlock });
        
        // Extract and update hidden inputs
        if (zone.dataAttributes && zone.dataAttributes.length > 0) {
            const data = getDataAttributes(item, zone.dataAttributes);
            for (const attr of zone.dataAttributes) {
                const inputId = (zone.hiddenInputPrefix || zoneId) + '-' + attr;
                updateHiddenInput(inputId, data[attr]);
            }
        }
        
        // Trigger callbacks
        if (triggerCallbacks) {
            if (zone.onFocusChange && typeof window[zone.onFocusChange] === 'function') {
                window[zone.onFocusChange](item, clampedIdx, zoneId);
            }
        }
    }
    
    notifyStateChange();
}

function setZoneFocus(zoneId) {
    const zone = getZoneConfig(zoneId);
    if (!zone) return;
    
    const container = getZoneElement(zoneId);
    if (container) {
        addFocusRing(container, zone.zoneFocusClasses);
    }
}
'''.strip()

## Zone Switching

In [ ]:
#| export
def js_zone_switching() -> str: # JavaScript zone switching code
    """Generate JavaScript code for zone switching."""
    return '''
// === Zone Switching ===
function setActiveZone(zoneId, triggerCallbacks = true) {
    const prevZoneId = activeZoneId;
    const prevZone = getZoneConfig(prevZoneId);
    const newZone = getZoneConfig(zoneId);
    
    if (!newZone) return;
    
    // Handle mode exit on zone change
    const currentModeConfig = getModeConfig(currentMode);
    if (currentModeConfig && currentModeConfig.exitOnZoneChange && currentMode !== cfg.defaultMode) {
        exitMode();
    }
    
    // Clear previous zone focus
    if (prevZone) {
        clearZoneFocus(prevZoneId);
        if (triggerCallbacks && prevZone.onZoneLeave && typeof window[prevZone.onZoneLeave] === 'function') {
            window[prevZone.onZoneLeave](prevZoneId);
        }
    }
    
    // Set new active zone
    activeZoneId = zoneId;
    setZoneFocus(zoneId);
    
    // Ensure item focus in new zone
    if (newZone.itemSelector) {
        setItemFocus(zoneId, focusIndices[zoneId] || 0, false);
    }
    
    // Trigger callbacks
    if (triggerCallbacks) {
        if (newZone.onZoneEnter && typeof window[newZone.onZoneEnter] === 'function') {
            window[newZone.onZoneEnter](zoneId);
        }
        if (cfg.callbacks.onZoneChange && typeof window[cfg.callbacks.onZoneChange] === 'function') {
            window[cfg.callbacks.onZoneChange](zoneId, prevZoneId);
        }
    }
    
    notifyStateChange();
}

function switchZone(direction) {
    const zoneIds = cfg.zones.map(z => z.id);
    const currentIdx = zoneIds.indexOf(activeZoneId);
    let newIdx;
    
    if (direction === 'next') {
        newIdx = currentIdx + 1;
        if (newIdx >= zoneIds.length) {
            newIdx = cfg.zoneSwitching.wrap ? 0 : zoneIds.length - 1;
        }
    } else {
        newIdx = currentIdx - 1;
        if (newIdx < 0) {
            newIdx = cfg.zoneSwitching.wrap ? zoneIds.length - 1 : 0;
        }
    }
    
    if (newIdx !== currentIdx) {
        setActiveZone(zoneIds[newIdx]);
    }
}
'''.strip()

## Navigation

In [ ]:
#| export
def js_navigation() -> str: # JavaScript navigation code
    """Generate JavaScript code for item navigation."""
    return '''
// === Navigation ===
function getNavigationPattern(zoneId) {
    // Check if mode overrides navigation
    const modeConfig = getModeConfig(currentMode);
    if (modeConfig && modeConfig.navigationOverride) {
        return modeConfig.navigationOverride;
    }
    // Use zone's pattern
    const zone = getZoneConfig(zoneId);
    return zone ? zone.navigationPattern : 'linear_vertical';
}

function navigate(direction) {
    const zone = getZoneConfig(activeZoneId);
    if (!zone || !zone.itemSelector) return false;
    
    const items = getZoneItems(activeZoneId);
    if (items.length === 0) return false;
    
    const pattern = getNavigationPattern(activeZoneId);
    const currentIdx = focusIndices[activeZoneId] || 0;
    let newIdx = currentIdx;
    
    // Calculate new index based on pattern
    if (pattern === 'linear_vertical') {
        if (direction === 'up') newIdx = Math.max(0, currentIdx - 1);
        else if (direction === 'down') newIdx = Math.min(items.length - 1, currentIdx + 1);
    } else if (pattern === 'linear_horizontal') {
        if (direction === 'left') newIdx = Math.max(0, currentIdx - 1);
        else if (direction === 'right') newIdx = Math.min(items.length - 1, currentIdx + 1);
    } else if (pattern === 'grid') {
        // Grid navigation will be implemented in future
        // For now, fall back to linear
        if (direction === 'up') newIdx = Math.max(0, currentIdx - 1);
        else if (direction === 'down') newIdx = Math.min(items.length - 1, currentIdx + 1);
    }
    
    if (newIdx !== currentIdx) {
        setItemFocus(activeZoneId, newIdx);
        
        // Trigger onNavigate callback
        if (zone.onNavigate && typeof window[zone.onNavigate] === 'function') {
            const item = getZoneItems(activeZoneId)[newIdx];
            window[zone.onNavigate](item, newIdx, activeZoneId, direction);
        }
        return true;
    }
    return false;
}
'''.strip()

## Mode Management

In [ ]:
#| export
def js_mode_management() -> str: # JavaScript mode management code
    """Generate JavaScript code for mode management."""
    return '''
// === Mode Management ===
function getModeConfig(modeName) {
    return cfg.modes.find(m => m.name === modeName);
}

function isModeAvailable(modeName) {
    const mode = getModeConfig(modeName);
    if (!mode) return false;
    if (!mode.zoneIds) return true;
    return mode.zoneIds.includes(activeZoneId);
}

function enterMode(modeName) {
    if (!isModeAvailable(modeName)) return false;
    
    const prevMode = currentMode;
    const mode = getModeConfig(modeName);
    
    // Exit current mode first
    if (prevMode !== cfg.defaultMode) {
        exitMode(false); // Don't notify yet
    }
    
    currentMode = modeName;
    
    // Trigger callbacks
    if (mode.onEnter && typeof window[mode.onEnter] === 'function') {
        window[mode.onEnter](modeName, activeZoneId);
    }
    if (cfg.callbacks.onModeChange && typeof window[cfg.callbacks.onModeChange] === 'function') {
        window[cfg.callbacks.onModeChange](modeName, prevMode);
    }
    
    notifyStateChange();
    return true;
}

function exitMode(notify = true) {
    const prevMode = currentMode;
    const mode = getModeConfig(prevMode);
    
    if (prevMode === cfg.defaultMode) return; // Can't exit default mode
    
    currentMode = cfg.defaultMode;
    
    // Trigger callbacks
    if (mode && mode.onExit && typeof window[mode.onExit] === 'function') {
        window[mode.onExit](prevMode, activeZoneId);
    }
    if (notify && cfg.callbacks.onModeChange && typeof window[cfg.callbacks.onModeChange] === 'function') {
        window[cfg.callbacks.onModeChange](cfg.defaultMode, prevMode);
    }
    
    if (notify) notifyStateChange();
}
'''.strip()

## Action Dispatch

In [ ]:
#| export
def js_action_dispatch() -> str: # JavaScript action dispatch code
    """Generate JavaScript code for action dispatch."""
    return '''
// === Action Dispatch ===
function actionMatchesContext(action) {
    // Check zone condition
    if (action.zoneIds && !action.zoneIds.includes(activeZoneId)) {
        return false;
    }
    // Check mode condition
    if (action.modeNames && !action.modeNames.includes(currentMode)) {
        return false;
    }
    // Check not_modes condition
    if (action.notModes && action.notModes.includes(currentMode)) {
        return false;
    }
    // Check custom condition
    if (action.customCondition) {
        try {
            if (!eval(action.customCondition)) return false;
        } catch (e) {
            console.warn('Custom condition failed:', e);
            return false;
        }
    }
    return true;
}

function executeAction(action) {
    // HTMX trigger
    if (action.htmxTrigger) {
        triggerClick(action.htmxTrigger);
    }
    // JS callback
    if (action.jsCallback && typeof window[action.jsCallback] === 'function') {
        const item = getFocusedItem(activeZoneId);
        window[action.jsCallback](item, focusIndices[activeZoneId], activeZoneId, currentMode);
    }
    // Mode enter
    if (action.modeEnter) {
        enterMode(action.modeEnter);
    }
    // Mode exit
    if (action.modeExit) {
        exitMode();
    }
}

function findMatchingAction(key, mods) {
    for (const action of cfg.actions) {
        if (action.key !== key) continue;
        if (!modifiersMatch(mods, action.modifiers)) continue;
        if (!actionMatchesContext(action)) continue;
        return action;
    }
    return null;
}
'''.strip()

## Keyboard Handler

In [ ]:
#| export
def js_keyboard_handler() -> str: # JavaScript keyboard handler code
    """Generate JavaScript code for keyboard event handling."""
    return '''
// === Keyboard Handler ===
function handleKeydown(e) {
    // Skip if input focused
    if (cfg.settings.skipWhenInputFocused && isInputFocused(e.target)) {
        return;
    }
    
    const key = e.key;
    const mods = getModifiers(e);
    
    // Check zone switching
    if (modifiersMatch(mods, cfg.zoneSwitching.modifiers)) {
        if (key === cfg.zoneSwitching.prevKey) {
            e.preventDefault();
            switchZone('prev');
            return;
        }
        if (key === cfg.zoneSwitching.nextKey) {
            e.preventDefault();
            switchZone('next');
            return;
        }
    }
    
    // Check mode entry/exit
    const currentModeConfig = getModeConfig(currentMode);
    
    // Check mode exit
    if (currentModeConfig && currentModeConfig.exitKey === key) {
        if (modifiersMatch(mods, currentModeConfig.exitModifiers || [])) {
            e.preventDefault();
            exitMode();
            return;
        }
    }
    
    // Check mode entry (from other modes)
    for (const mode of cfg.modes) {
        if (mode.enterKey === key && mode.name !== currentMode) {
            if (modifiersMatch(mods, mode.enterModifiers || [])) {
                if (isModeAvailable(mode.name)) {
                    e.preventDefault();
                    enterMode(mode.name);
                    return;
                }
            }
        }
    }
    
    // Check navigation (only if no modifiers for zone switch)
    const direction = cfg.keyMapping[key];
    if (direction && mods.size === 0) {
        const zone = getZoneConfig(activeZoneId);
        if (zone && zone.itemSelector) {
            if (navigate(direction)) {
                e.preventDefault();
                return;
            }
        }
    }
    
    // Check actions
    const action = findMatchingAction(key, mods);
    if (action) {
        if (action.preventDefault) e.preventDefault();
        if (action.stopPropagation) e.stopPropagation();
        executeAction(action);
        return;
    }
}
'''.strip()

## State Notification

In [ ]:
#| export
def js_state_notification() -> str: # JavaScript state notification code
    """Generate JavaScript code for state change notification."""
    return '''
// === State Notification ===
function getState() {
    return {
        activeZoneId: activeZoneId,
        focusIndices: {...focusIndices},
        currentMode: currentMode
    };
}

function notifyStateChange() {
    // Expose globally if configured
    if (cfg.settings.exposeStateGlobally) {
        window[cfg.settings.globalStateName] = getState();
    }
    
    // Update hidden inputs if configured
    if (cfg.settings.stateHiddenInputs) {
        updateHiddenInput('kb-active-zone', activeZoneId);
        updateHiddenInput('kb-current-mode', currentMode);
        updateHiddenInput('kb-focus-indices', JSON.stringify(focusIndices));
    }
    
    // Trigger callback
    if (cfg.callbacks.onStateChange && typeof window[cfg.callbacks.onStateChange] === 'function') {
        window[cfg.callbacks.onStateChange](getState());
    }
}
'''.strip()

## Initialization

After HTMX swaps, the DOM changes but our state (focus indices) may be stale. The initialization function:
1. Reads the hidden input values (set before the swap)
2. Finds items with matching data attributes in the new DOM
3. Updates focus indices to follow moved/reordered items
4. Falls back to clamping if item not found (deleted)

In [ ]:
#| export
def js_initialization() -> str: # JavaScript initialization code
    """Generate JavaScript code for initialization with focus recovery."""
    return '''
// === Focus Recovery ===
function findItemByDataAttribute(zoneId, attrName, attrValue) {
    // Find an item in the zone that has the specified data attribute value
    if (!attrValue) return -1;
    
    const items = getZoneItems(zoneId);
    for (let i = 0; i < items.length; i++) {
        const itemValue = items[i].getAttribute('data-' + attrName);
        if (itemValue === attrValue) {
            return i;
        }
    }
    return -1;
}

function recoverFocusForZone(zoneId) {
    // Try to find the previously focused item by its data attributes
    const zone = getZoneConfig(zoneId);
    if (!zone || !zone.dataAttributes || zone.dataAttributes.length === 0) {
        return false;
    }
    
    const items = getZoneItems(zoneId);
    if (items.length === 0) return false;
    
    // Try each data attribute to find the item
    for (const attr of zone.dataAttributes) {
        const inputId = zone.hiddenInputPrefix + '-' + attr;
        const input = document.getElementById(inputId);
        if (input && input.value) {
            const foundIdx = findItemByDataAttribute(zoneId, attr, input.value);
            if (foundIdx >= 0) {
                focusIndices[zoneId] = foundIdx;
                return true;
            }
        }
    }
    
    return false;
}

// === Initialization ===
function initialize() {
    for (const zone of cfg.zones) {
        const items = getZoneItems(zone.id);
        
        if (items.length > 0) {
            // Try to recover focus by finding the item with saved data attribute
            const recovered = recoverFocusForZone(zone.id);
            
            if (!recovered) {
                // Item not found (deleted?), clamp index to bounds
                focusIndices[zone.id] = Math.max(0, Math.min(
                    focusIndices[zone.id] || 0,
                    items.length - 1
                ));
            }
        } else {
            // No items, reset to 0
            focusIndices[zone.id] = 0;
        }
    }
    
    // Set active zone and apply focus
    setActiveZone(activeZoneId, false);
    
    // Initialize state notification
    notifyStateChange();
}

// === Event Listeners ===
document.addEventListener('keydown', handleKeydown);
document.body.addEventListener(cfg.settings.htmxSettleEvent, initialize);

// Initial setup
initialize();
'''.strip()

## Complete Script Generation

In [ ]:
#| export
def generate_keyboard_script(
    manager: ZoneManager  # the zone manager configuration
) -> str:                 # complete JavaScript code wrapped in IIFE
    """Generate complete keyboard navigation JavaScript from ZoneManager."""
    config = manager.to_js_config()
    
    parts = [
        "(function() {",
        "'use strict';",
        "",
        js_config_from_dict(config),
        "",
        js_all_utils(
            input_selector=manager.input_selector,
        ),
        "",
        js_zone_state(),
        "",
        js_focus_management(),
        "",
        js_zone_switching(),
        "",
        js_navigation(),
        "",
        js_mode_management(),
        "",
        js_action_dispatch(),
        "",
        js_state_notification(),
        "",
        js_keyboard_handler(),
        "",
        js_initialization(),
        "",
        "})();",
    ]
    
    return "\n".join(parts)

In [ ]:
# Test complete script generation
from cjm_fasthtml_keyboard_navigation.core.focus_zone import FocusZone
from cjm_fasthtml_keyboard_navigation.core.actions import KeyAction

browser = FocusZone(
    id="browser",
    item_selector="tr.item",
    data_attributes=("job-id",)
)
queue = FocusZone(
    id="queue",
    item_selector="li.item"
)

manager = ZoneManager(
    zones=(browser, queue),
    actions=(
        KeyAction(key=" ", htmx_trigger="toggle-btn"),
        KeyAction(key="Delete", htmx_trigger="delete-btn"),
    )
)

script = generate_keyboard_script(manager)

# Verify key parts are present
assert "(function()" in script
assert "const cfg" in script
assert "isInputFocused" in script
assert "handleKeydown" in script
assert "initialize" in script
assert "})();" in script

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()